In [41]:
import string
from random import random

import pandas as pd
import yfinance as yf
from IPython.core.display import display
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import random
import string
from random import randrange

In [ ]:
# Downloads 90 day's worth of data

Cron job will be set for Monday to Friday
1. Iterate for all the stocks
2. Check for any latest stock price
3. Send the data to elastic


In [56]:
stocks_list = ["AAPL", "TSLA", "GM", "MSFT"]
index_1m = "us_tickers_1m"

## Minute Data downloader

In [57]:
# data = yf.download(
#     tickers=stocks_list,
#     period="1d",
#     interval="1m",
#     group_by = 'ticker',
#     auto_adjust = True,
#     threads = True
# )

es_client = Elasticsearch(['localhost'], port=9200, http_compress=True)

[*********************100%***********************]  4 of 4 completed


In [79]:
# def random_id_generator():
#     id_size = randrange(5,10)
#     return ''.join(random.choices(string.ascii_uppercase + string.digits, k=id_size))
#
# def doc_generator(df, ticker_name):
#     df_iter = df.iterrows()
#     for df_idx, document in df_iter:
#         doc_dict = document.to_dict()
#         doc_dict["ticker"] = ticker_name
#         doc_id= str(document["Datetime"].value) + "-" + str(random_id_generator())
#         final_doc = {
#                 "_index": index_1m,
#                 "_type": "_doc",
#                 "_id" : f"{doc_id}",
#                 "_source": doc_dict,
#             }
#
#         yield final_doc

In [54]:
# for item in stocks_list:
#     # Clear the index
#     res_data = data[item].reset_index(level=0)
#     # Doc Generator
#     doc_generator(res_data, item)
#     # ES Writer
#     helpers.bulk(es_client, doc_generator(res_data, item))
#     break

## Close data downloader

In [82]:
start_date = "2021-05-07"
end_date = "2021-05-08"
interval = "1d"

In [74]:
# data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-05-06,127.889999,129.750000,127.129997,129.740005,129.520004,78128300
1,2021-05-07,130.850006,131.259995,129.479996,130.210007,130.210007,78892700


In [100]:
def download_1d_data(ticker, s_date, e_data):
    """
    Download 1 day\'s worth of data from start to end data
    :param ticker: Ticker name , e.g: "AAPL"
    :param s_date: Start date, e.g: "2021-05-07"
    :param e_data: End date, e.g: "2021-05-07"
    :return:
    """
    data_interval = "1d"
    data_1_day = yf.download(tickers=ticker,
                   start=s_date,
                   end=e_data,
                   interval=data_interval)
    data_1_day.reset_index(level=0, inplace=True)
    display(data_1_day.dtypes)
    return data_1_day

def download_min_data(ticker, s_date, e_date):
    data_interval = "1m"
    data_1_min = yf.download(
        tickers=ticker,
        start=s_date,
        end=e_date,
        interval=data_interval,
        auto_adjust = True,
        threads = True
    )
    data_1_min.reset_index(level=0, inplace=True)
    return data_1_min

def random_id_generator():
    id_size = randrange(5,10)
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=id_size))

def doc_generator(df, ticker_name, date_time_field, es_index):
    df_iter = df.iterrows()
    for df_idx, document in df_iter:
        doc_dict = document.to_dict()
        doc_dict["ticker"] = ticker_name
        doc_id= str(document[date_time_field].value) + "-" + str(random_id_generator())
        final_doc = {
                "_index": es_index,
                "_type": "_doc",
                "_id" : f"{doc_id}",
                "_source": doc_dict,
            }

        yield final_doc

In [96]:
stocks_list = ["TSLA", "GM", "MSFT"]
index_1_min = "us_tickers_1m"
index_1_day = "us_tickers_1d"

In [97]:
for item in stocks_list:
    # Download the data
    res_data = download_1d_data(item, start_date, end_date)
    # Doc Generator
    # doc_generator(res_data, item, index_1_day)
    # ES Writer
    helpers.bulk(es_client, doc_generator(res_data, item, "Date", index_1_day))
    break

[*********************100%***********************]  1 of 1 completed


Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object

d:\workspace\stock\finance_dataset_downloader\venv\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [98]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2021-05-08'

In [104]:
d = download_min_data("AAPL", start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [105]:
d

,Datetime,Open,High,Low,Close,Volume
0,2021-05-07 09:30:00-04:00,130.129105,130.160004,130.059998,130.089996,3751200
1,2021-05-07 09:31:00-04:00,130.089996,130.289993,129.899994,129.899994,555927
2,2021-05-07 09:32:00-04:00,129.899994,130.160004,129.820007,130.009995,433729
3,2021-05-07 09:33:00-04:00,130.011993,130.179993,129.989700,130.053497,354315
4,2021-05-07 09:34:00-04:00,130.039993,130.080002,129.750000,130.009903,415355
...,...,...,...,...,...,...
385,2021-05-07 15:55:00-04:00,130.455002,130.550003,130.449997,130.500000,360894
386,2021-05-07 15:56:00-04:00,130.500000,130.500000,130.320007,130.339996,372397
387,2021-05-07 15:57:00-04:00,130.339996,130.429993,130.330093,130.425003,274799
388,2021-05-07 15:58:00-04:00,130.420105,130.429993,130.229996,130.240097,503093
